# COUNT-CP, a CP constraint learner for the holy grail 2021 challenge
## Mohit Kumar, Samuel Kolb, Tias Guns

### KU Leuven, Belgium

__Abstract__: We introduce the COUNT-CP constraint learner for CP problems.
Count-CP is based on the idea behind Mohit Kumar et al's COUNT-OR, a constraint learning algorithm for personnel rostering problems: empirically finding the suitable upper and lower bounds for relevant expressions `lb <= expr <= ub`.
COUNT-OR learns bounds for expressions that occur frequently in rostering problems: aggregation operations on slices of tensors.
 In contrast, COUNT-CP uses a grammar over decision variables or pairs of decision variables suitable for expressing constraints typical for CP problems.
Additionally, COUNT-CP introduces an additional step for generalizing learned models across different instance sizes.
COUNT-CP is implemented using the CPMpy constraint programming and modeling environment.

Our proposal consists of two parts:
 - Part 1: learning constraints for instances of the same size
 - Part 2: learning generalized constraint models

## Part 1: learning constraints for instances of the same size

### Constraint bias

Previous work [(Kumar et al.)](https://ieeexplore.ieee.org/abstract/document/8995316) on learning for personnel rostering focussed on learning constraints over expressions that aggregate over various slices of a tensor, e.g.,

```
foreach i, j:
    0 <= sum(X[i, j, :]) <= 1
```

In this work, we consider the case of learning constraints of the following form:
```
lb <= expr <= ub
```
where `expr` is a numeric expression over one or two decision variables, such as `x + y` or `abs(x-y)`.
To learn these constraints, we defined a grammar that captures expressions frequently occurring in CP problems.


### Expression grammar

We consider both _unary_ expressions, that is expression involving one individual variable, and _binary_ expressions involving two variables.

To construct our grammar, we look at unary and binary expressions that can be expressed in the generic CPMpy modeling language. The expressions are the same for other modeling languages.

We consider the unary operators identity and absolute value, as well as the binary operators addition and subtraction.
Unary expressions are generated using unary operators, while binary expressions are generated by combining both unary and binary operators.  More concretely, we have the following grammar rules:

In [ ]:
# [x, abs(x)]
def unary_operators():
    yield lambda x: x  # identity
    yield abs

# [x+y, x-y, y-x, |x|+|y|, |x|-|y|, |y|-|x|]
def binary_operators():
    for u in unary_operators():
        yield lambda x, y: u(x) + u(y)
        yield lambda x, y: u(x) - u(y)
        yield lambda x, y: u(y) - u(x)

# the unary expressions = unary operators
def generate_unary_expr(x):
    for u in unary_operators():
        yield u(x)

# the binary expressions = unary operators wrapped around binary
# e.g. for binary x-y: x-y, abs(x-y), abs(abs(x)-abs(y))
def generate_binary_expr(x, y):
    for b in binary_operators():
        for u in unary_operators(): # includes identity
            yield u(b(x, y))

Observe how this does not include the 'traditional' constraint biases `x != y`, `x <= y`, `x < y`, etc. The reason is that we have expressions that subsume those, namely correspondingly `abs(x - y) >= 1`, `x - y <= 0` and `x - y <= -1`. Hence, the above constraint bias (the inequalities over that grammar) can learn those and more.

One of the few unary/binary constraints it can not learn is `x != c`,
for some constant `c` which lies between (exclusive) the lower and upperbound of `x`. We believe it will be very rare that a constraint model intentionally excludes one individual value, without that value being specified as the 'input data'. Hence, it is not part of our bias.

The bias also does not include n-ary global constraints such as `alldifferent()` or `increasing()`, however, these have decompositions into binary constraints, meaning that we can learn the decomposed versions.
Also not included are tertiary constraints such as `x + y = z` or, equivalently, bounds on `x + y - z` for arbitrary triples. We leave it open whether these constructs are commonly used, and how to best manage the large number of candidates.

Finally, we also don't support element constraints like `list[x] == y`; they typically interact between different variable matrices (e.g. `list` and some variables `x`,`y`), for which we have not yet defined a grammar.

For this challenge this simple grammar already allowed us to learn the large majority of constraints.  However, for more complicated problems, the grammar can trivially be extended with additional unary (e.g., `x*x`, `mod(x, 2)`), binary or n-ary operators (e.g., sums over lists).  This increased expressiveness would, however, incur an additional computational cost during learning.

### Constraint learning

Our learning algorithm only makes use of __positive__ data, that is, the given true solutions. As all solutions have the same size, we can easily represent them as a tensor. More specifically, let there be `N` examples, where each example is a list of size `d`, then we represent the positive instances as an `N x d` matrix.

Similarly, for `N x d1 x d2` for an `d1 x d2` shaped matrix of decision variables. Without loss of generality, we can represent it as an `N x d` matrix, with `d = d1 x d2`, that is, a flattened matrix in which every row is an example and every column is one element of the `d1 x d2` shaped matrix of decision variables.
If solutions contain multiple sets of decision variables, we can convert them into `N x d` matrix separately and concatenate these matrices.
For example, if every solution contains two sets of decision variables of size `d1` and `d2`, respectively, we represent the solutions by two matrices of size `N x d1` and `N x d2` and concatenate them to obtain an `N x d` matrix where `d = d1 + d2`.
From now on we assume an `N x d` representation of the solutions.

We will generate all possible expressions from the grammar, apply it on all possible elements of the matrices, and compute the resulting bounds. This can be seen as a convex hull of all possible expressions around the given solutions. This is a valid, though potentially overly restrictive, representation of constraints which the data satisfies.

Our approach has four phases:

1. Generate and compute expressions
2. Translate bounded expressions to constraints
3. Remove implied constraints
4. Learning the objective function

#### 1. Generate and compute expressions

Given a tensor (list, matrix, or higher dimensionality) of decision variables, we  generate all possible unary expressions, and for each expression, we apply it on each of the columns of our corresponding `N x d` matrix of solutions.

For every pair of expression `expr` and column index `i`, we obtain a list of `N` values and can easily compute the min and max values.
This way, we obtain a data structure with tuples `(expr, i, lb, ub)`, where `lb` and `ub` are the lower and upper bounds of an expression applied to a decision variable `i` across all positive examples.

Next, we do the same for binary expressions over all _pairs_ of columns in the `N x d` solution matrix, and we compute and store the bounds of the resulting lists of values.
In the resulting data structure we then store the bounds as `(expr, (i1, i2), lb, ub)`, where `i1` and `i2` are the indices of the two decision variables (or columns) to which the expression was applied.

(From a technical point of view, unary index tuples `(expr, i, lb, ub)` are represented as `(expr, (i,), lb, ub)`.  This representation also allows extensions to n-ary expressions `(expr, (i1, i2, ..., in), lb, ub)`.)

Currently, constraints are computed for each set of decision variables separately.
For problem type `03`, this means that we do not learn binary constraints that mix the `customer` and `warehouse` decision variables.
We imposed this restriction to decrease the computation time for our experiments, however, removing this restriction allows more expressive constraints to be learned.

#### 2. Translate bounded expressions to constraints

We read in the challenge data, extract from the `formatTemplate` the number of list/matrix variables and their shape, as well as their lower and upper bound.

Next, we create corresponding CPMpy decision variables with corresponding domains.

On those, we can then apply the same expressions as used in step 1. CPMpy overloads Python operators, so applying the operators results in valid CPMpy expressions and constraints. However, for many expressions, the computed lower or upper bound is trivial: it is equivalent to the lower or upper bound of the expression given the domains of the variables.

To detect these trivial constraints with non-restricting bounds, we can make use of CPMpy's mechanism to create auxiliary variables with tight domains for arbitrary expressions. Only if the learned bounds are different from those computed by CPMpy for the generic expression, the corresponding constraint is added to a candidate model.

Note that this does not change the set of solutions covered, it just makes it easier to inspect the models and slightly more efficient to execute it for the solver.

#### 3. Remove implied constraints

While we already removed constraints with trivial bounds, we can still have implied constraints, such as `abs(x + y) >= 2` and `x + y >= 2` while x and y can only take positive values. More complex examples exist.

We consider a constraint `c1` to be _implied by_ a constraint `c2` if all solutions of `c2` are also solutions of c1. That is, `c1` constrains a subset (or equal) of `c2`. Logically, we have `c2 -> c1`. This means that `not (c2 -> c1)` == `c2 and not c1` can not be true. In other words, we can use a constraint solver to search for a counter example for which `c2 and not c1`. If it finds one, the `c1` is not implied by `c2`. If it finds no such solution, then `c1` is implied, and it can safely be removed. This will again not change the set of accepted solutions, but it will make the learned constraint models more interpretable.

More generally, for a set of learned constraints `C`, we check for each one in turn whether it is implied by the other constraints or not, and remove it if it is. The pseudo-code is as follows:

In [ ]:
C = [...] # list of constraints
i = 0
while i < len(C):
    m = Model(C[:i] + C[i+1:] + ~all(C[i]))
    if m.solve():
        # skip it, not implied
        i += 1
    else:
        del C[i]

where `C[:i] + C[i+1:]` consists of all constraints except `C[i]` and `~all(C[i])` is the negation of `C[i]` (even when it is a list of constraints).

With a lazy clause generation solver like ortools CP-SAT, generating (or disproving) such counter examples is pretty efficient.

Note that the _order_ in which the constraints are tested for removal matters... We assume that the grammar rules are ordered from 'simple' to 'complex', and hence we traverse the list of constraints from the end (more complex) to the beginning (the unary constraints, simple).

Improving this ordering of constraints through more elaborate estimates of which constraint should be preferred over other equivalent ones could be useful.

#### 4. Learning the objective function

We feel the challenge has a rather loose interpretation of 'objective function', given that there are multiple solutions given and hence it does not seem to be a function that has to be optimized.

Instead, one could see learning the objective function as an _equation discovery_ problem: given a solution, what equation generates the given value.
Equations could be generated using a simple grammar of n-ary expressions in CPMpy, i.e., `sum, min, max`.
However, in this challenge, the objective seems to be simply `max` in all types with objective functions except one.

The other one, type `03`, involves weighted sums over projected variable representations. We could add weighted sums over matching matrix dimensions, but with the limited time available we decided to manually provide the required objective function for this problem.


### Results for local learning
The following table shows the results for this local learning approach:

In [2]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option('precision', 2)
df = pd.read_csv("merged.csv")
names = df['file'].str.split('/', n=2, expand=True)
df['type'] = names[1]
df['inst'] = names[2]
df['percentage_pos'] = df['percentage_pos'].round(2)
df['percentage_neg'] = df['percentage_neg'].round(2)
df[['type','inst','constraints','filtered_constraints','percentage_pos', 'percentage_neg']].groupby(['type']).mean()

,constraints,filtered_constraints,percentage_pos,percentage_neg
type,,,,
type01,252.25,33.38,100.0,100.00
type02,277.50,35.88,100.0,99.91
type04,506.00,21.50,100.0,99.68
type07,82.18,5.82,100.0,77.63
type08,426.75,84.62,100.0,99.93
type13,155.78,20.33,100.0,99.50
type14,868.67,98.67,100.0,100.00
type15,140.40,18.80,100.0,99.95
type16,651.11,89.11,100.0,100.00


We can see in the above table that the reduction from initial set of constraints to filtered (non-trivial, non-implied) set of constraints is very large.
In part, the initial set of constraints could already be reduced by pre-processing expressions in the grammar, e.g., for positive variables `x` and `y`, expressions `x + y` and `abs(x + y)` are equivalent and one of them can be excluded before learning.

As per construction, our approach always satisfies all positive examples.

The last column shows the percentage of negative examples that we _reject_. That is, our constraint model correctly answers 'UNSAT' when we force the decision variables to equal the negative example.

It shows that our local constraint learning approach achieves >99% accuracy on many constraint types, with a reasonable number of constraints in the model. The exception is `type07`, where a limited visual inspection did not reveal to the authors what a missing constraint could be.

__Also not shown in this table are type03, type05, type06, type10, type11 and type12. These involve more then one decision variable, which Samuel still has to implement : P__

## Part 2: learning generalized constraint models

- generalize same constraints over index groups
    -> generate constraint groups, check bounds
    -> trivial bound check
- implication check
- delete additional constraints
- FW: symbolic bounds on bounds

In [5]:
df = pd.read_csv("type_level_results.csv")
names = df['file'].str.split('/', n=2, expand=True)
df['type'] = names[1]
df['inst'] = names[2]
df['percentage_pos'] = df['percentage_pos'].round(2)
df['percentage_neg'] = df['percentage_neg'].round(2)
df[['type','inst','constraints','percentage_pos', 'percentage_neg']].groupby(['type']).mean()

,constraints,percentage_pos,percentage_neg
type,,,
type01,0,100,90.74
type02,2,100,99.85
type04,3,100,96.66
type07,0,100,0.00
type08,3,100,98.25
type13,1,100,98.34
type14,4,100,100.00
type15,2,100,99.16
type16,2,100,100.00


Weaker results, esp on Type01 and Type07.

__However, it should be noted that these were learned on ?one? and evaluated on all others.??__

Note that they can also be applied on the instances for which no positive/negative examples are given.

# Conclusion

We implemented a constraint learner through an inequality-based learner over a constraint grammar involving unary and binary expressions over decision variables.
Key techniques used are grammars, matrix operations to efficiently extract bounds, and the CPMpy modeling environment.

Our local learning approach achieves high accuracy and automatically reduces the size of the learned model by removing trivial and implied constraints.
We also investigated automated ways to generalize the learned constraints, with decent initial success.

Much future work remains, especially with respect to reducing the set of local constraints further to an 'elementary' set; as well as generalizing constraints across instances of different size.